# Імпорти

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install torch-directml

In [ ]:
!pip install tensorflow-directml-plugin

In [ ]:
!pip install kaggle

In [ ]:
!pip install pandas

In [ ]:
!pip install plotly

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go

from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras_tuner import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Набір дних, попередня підготовка даних

## Завантаження з Kaggle

Завантажимо набір даних для компанії через Kaggle API. Компанія CNQ (Canadian Natural Resources Limited Common Stock).

Спочатку треба завантажити токен до Kaggle API, аби пройти автентифікацію. Завантажимо з Kaggle на комп'ютер, потім на Google Drive, далі з диску у Colab.

Підключились до диску:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Завантажили в Colab файл із токеном:

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Увійшли в Kaggle API:

In [ ]:
api = KaggleApi()
api.authenticate()

Завантажили весь набір даних:

In [ ]:
api.dataset_download_files('jacksoncrow/stock-market-dataset', path='./data', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset


Завантажили дані для компанії CNQ, stock її:

In [ ]:
df = pd.read_csv('data/stocks/CNQ.csv')

In [ ]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-07-31,3.56250,3.562500,3.484375,3.484375,1.778393,104000.0
1,2000-08-01,3.50000,3.500000,3.429688,3.500000,1.786368,62400.0
2,2000-08-02,3.62500,3.679688,3.625000,3.656250,1.866116,46400.0
3,2000-08-03,3.65625,3.710938,3.648438,3.664062,1.870104,74400.0
4,2000-08-04,3.65625,3.679688,3.625000,3.671875,1.874091,61600.0
...,...,...,...,...,...,...,...
4945,2020-03-26,10.10000,11.390000,10.010000,10.410000,10.410000,10156300.0
4946,2020-03-27,9.78000,9.960000,9.130000,9.490000,9.490000,4681200.0
4947,2020-03-30,9.24000,11.270000,8.950000,11.130000,11.130000,12264700.0
4948,2020-03-31,11.49000,13.820000,11.490000,13.550000,13.550000,23796600.0


## Нормалізація

Функція, що створить "windowed" набір даних, який зручний для тренування моделей на часових рядах з використанням window-методу.<br>
Параметри:
- data: вхідний масив даних (історичні фінансові дані), заданий часовим рядом.
- window_size: розмір вікна (кількість послідовних записів у кожному вхідному зразку).
- forecast_days: кількість днів у майбутньому для прогнозування (цільовий зсув).
- step: крок, з яким обираються наступні вікна (за замовчуванням 1, тобто обирається кожне наступне вікно).
<br>Повертає:
- X: вхідні дані (масив вікон).
- y: цільові значення (масив прогнозів).

In [ ]:
def create_windowed_dataset(data, window_size, forecast_days, step=1):
    X = []
    y = []

    for i in range(0, len(data) - window_size - forecast_days, step):
        # створюємо вхідні вікна довжиною window_size
        window_data = data[i:i + window_size]
        X.append(window_data)

        # визначаємо цільове значення, яке є forecast_days днів після кінця вікна
        target_index = i + window_size + forecast_days - 1
        target_value = data[target_index][4]  # цільове значення Close знаходиться в п'ятому стовпчику
        y.append(target_value)

    return np.array(X), np.array(y)

In [ ]:
features = df[['Open', 'High', 'Low', 'Volume']]  # вхідні
target = df['Close']  # цільова колонка

Також scaler для зміни розміру features:

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaled_features = scaler.fit_transform(features)

In [ ]:
combined_data = np.hstack([scaled_features, target.values.reshape(-1, 1)])

Є NaN значення, тому їх видалимо:

In [ ]:
total_nans = np.isnan(combined_data).sum()
print(f"Загальна кількість NaN значень у датасеті: {total_nans}")

Загальна кількість NaN значень у датасеті: 5


In [ ]:
# видалення рядків з NaN значеннями
combined_data = combined_data[~np.isnan(combined_data).any(axis=1)]

# перевірка, що всі NaN значення видалені
total_nans_after = np.isnan(combined_data).sum()
print(f"Загальна кількість NaN значень після очищення: {total_nans_after}")

Загальна кількість NaN значень після очищення: 0


## Поділ набору даних на тренувальну та тестувальну вибірки

Створюємо набір через вищеоголошену функцію:

In [ ]:
# параметри для функції
wn_size = 4
ft_days = 1
step = 1
X, y = create_windowed_dataset(combined_data, wn_size, ft_days, step=step)  # створюємо набір
# додатково масив індексів
indexes = np.arange(len(X))

Поділ даних:

In [ ]:
X_train, X_test, y_train, y_test, X_i, y_i = train_test_split(X, y, indexes, test_size=0.2, shuffle=True, )

Перевірка, чи є NaN-и:

In [ ]:
print(np.isnan(X_train).sum(), np.isnan(y_train).sum())
print(np.isinf(X_train).sum(), np.isinf(y_train).sum())

0 0
0 0


## Функції для тренування, валідації (тестування) моделей, побудови графіків

Тренування:

In [ ]:
def train_model(model, epochs):
    batch_size = 32
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model.name+'best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
    if epochs == 1:
        model.fit(X_train, y_train)
    else:
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[checkpoint])
        model = tf.keras.models.load_model(model.name+'best_model.h5')

Валідація:

In [ ]:
def validate_model(model):
    loss = model.evaluate(X_test, y_test)
    print(f'Test loss: {loss}')

Для графіків:

In [ ]:
def plot_predictions(model, predictions=None):

    if predictions is not None:
      predictions = predictions
    else:
      predictions = model.predict(X_test)

    # дати для тестових даних
    dates_test = [df['Date'][i * step + wn_size + ft_days - 1] for i in y_i]

    # DataFrame для порівняння фактичних та прогнозованих значень
    comparison_dt = pd.DataFrame({'Actual': y_test, 'Predicted': predictions.flatten(), 'Date': dates_test}).sort_values(
        by='Date')
    print(comparison_dt)

    # графік
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comparison_dt['Date'], y=comparison_dt['Actual'], mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=comparison_dt['Date'], y=comparison_dt['Predicted'], mode='lines', name='Predicted'))
    fig.update_layout(title='Stock Prices: actual / predicted', xaxis_title='Date',yaxis_title='Stock price')

    fig.show()

In [ ]:
def train_and_plot(model, epochs=100):
    train_model(model, epochs)
    validate_model(model)
    plot_predictions(model)

# Linear Regression (Лінійна регресія)

Модель:

In [ ]:
model_LR = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(1)
])
model_LR.compile(optimizer='adam', loss='mse')

Навчання, валідація, графік:

In [ ]:
train_model(model_LR, epochs=200)

Epoch 1/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4753 - val_loss: 0.4960
Epoch 2/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4731 - val_loss: 0.5164
Epoch 3/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4781 - val_loss: 0.5020
Epoch 4/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4718 - val_loss: 0.4973
Epoch 5/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4737 - val_loss: 0.4954
Epoch 6/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4763 - val_loss: 0.4982
Epoch 7/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4683 - val_loss: 0.4945
Epoch 8/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4736 - val_loss: 0.5024
Epoch 9/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4752 - val_loss: 0.5007
Epoch 10/200
124/124 [==============================] - 0s 2ms/step - loss: 0.4754 - val_lo

In [ ]:
validate_model(model_LR)

31/31 [==============================] - 0s 1ms/step - loss: 0.4977
Test loss: 0.49768131971359253


In [ ]:
plot_predictions(model_LR)

31/31 [==============================] - 0s 966us/step
        Actual  Predicted        Date
972   3.859375   3.846120  2000-08-14
224   3.945312   3.996812  2000-08-22
722   4.078125   4.025285  2000-09-05
260   4.187500   4.102266  2000-09-06
877   4.218750   4.199906  2000-09-11
..         ...        ...         ...
748  24.250000  24.704626  2020-03-04
298  15.730000  22.566792  2020-03-06
631  15.080000  15.934489  2020-03-10
849  14.380000  12.601683  2020-03-12
46   10.410000  10.097647  2020-03-25

[989 rows x 3 columns]


## Linear Regression (вбудована лінійна регресія)

Модель:

In [ ]:
model_LR2 = LinearRegression()

# перетворимо тренувальні дані в 1-вимірний масив через метод flatten
flatten_train_data = []
for x_train_data in X_train:
    flatten_train_data.append(x_train_data.flatten())

Тренуємо:

In [ ]:
model_LR2.fit(flatten_train_data, y_train)

LinearRegression()

Передбачення:

In [ ]:
# передбачення теж на основі перетворених в 1-вимірний масив даних X_test
y_preds = []
for x_test_data in X_test:
    y_preds.append(model_LR2.predict([x_test_data.flatten()]))

Оцінка:

In [ ]:
print("Mean Squared Error, MSE:", mean_squared_error(y_test, y_preds))
print("Mean Absolute Error, MAE:", mean_absolute_error(y_test, y_preds))
print("R^2 score:", r2_score(y_test, y_preds))

Mean Squared Error, MSE: 0.5028052652852383
Mean Absolute Error, MAE: 0.4677287562536077
R^2 score: 0.9968119199317811


Графік:

In [ ]:
plot_predictions(model_LR2, predictions=np.array(y_preds))

        Actual  Predicted        Date
972   3.859375   3.839580  2000-08-14
224   3.945312   3.986114  2000-08-22
722   4.078125   4.018417  2000-09-05
260   4.187500   4.098992  2000-09-06
877   4.218750   4.194945  2000-09-11
..         ...        ...         ...
748  24.250000  24.639929  2020-03-04
298  15.730000  22.567965  2020-03-06
631  15.080000  15.805296  2020-03-10
849  14.380000  11.942930  2020-03-12
46   10.410000  10.267389  2020-03-25

[989 rows x 3 columns]


# Random Forest Regressor (Випадковий ліс для регресії)

Як вбудована лінійна регресія, через вбудовану функцію.

In [ ]:
model_RF = RandomForestRegressor()

І також тренувальні/тестувальні дані мають бути приведені до 1-вимірного масиву, тому передаємо flatten_train_data.

In [ ]:
model_RF.fit(flatten_train_data, y_train)

RandomForestRegressor()

Аналогічно, передбачення (з приведених до 1-вимірного масиву даних X_test):

In [ ]:
y_preds = []
for x_test_data in X_test:
    y_preds.append(model_RF.predict([x_test_data.flatten()]))

Оцінка:

In [ ]:
print("Mean Squared Error, MSE:", mean_squared_error(y_test, y_preds))
print("Mean Absolute Error, MAE:", mean_absolute_error(y_test, y_preds))
print("R^2 score:", r2_score(y_test, y_preds))

Mean Squared Error, MSE: 0.482602824518735
Mean Absolute Error, MAE: 0.444975274829472
R^2 score: 0.9957593837217403


Як бачимо, випадковий ліс трохи кращий за лінійну регресію.<br>
**ЛР:** MSE: 0.503 MAE: 0.46 R^2 score: 0.997<br>
**ВЛ:** MSE: 0.482 MAE: 0.45 R^2 score: 0.996


Графік:

In [ ]:
plot_predictions(model_RF, predictions=np.array(y_preds))

        Actual  Predicted        Date
972   3.859375   3.819284  2000-08-14
224   3.945312   3.992884  2000-08-22
722   4.078125   4.021484  2000-09-05
260   4.187500   4.041591  2000-09-06
877   4.218750   4.175069  2000-09-11
..         ...        ...         ...
748  24.250000  24.803800  2020-03-04
298  15.730000  22.557100  2020-03-06
631  15.080000  15.822550  2020-03-10
849  14.380000  12.272775  2020-03-12
46   10.410000  10.064275  2020-03-25

[989 rows x 3 columns]


# LSTM

## Підбір найкращих гіперпараметрів

 build_model creates an LSTM-based recurrent neural network model, and then using the Keras Tuner's BayesianOptimization to search for the optimal hyperparameters for the model. The purpose of this process is to find the best model architecture and hyperparameters to improve the model's performance on your specific task.

In [ ]:
import tensorflow as tf
from keras_tuner import BayesianOptimization

# Функція побудови моделі з гіперпараметрами
def build_model(hp):
    model = tf.keras.models.Sequential([
        # Перший шар RNN з налаштуванням кількості одиниць
        tf.keras.layers.RNN(
            tf.keras.layers.LSTMCell(hp.Int('units', min_value=32, max_value=256, step=16), implementation=2),
            return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        # Dropout шар для регуляризації
        tf.keras.layers.Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)),
        # Другий шар RNN з налаштуванням кількості одиниць
        tf.keras.layers.RNN(
            tf.keras.layers.LSTMCell(hp.Int('units2', min_value=32, max_value=256, step=16), implementation=2)),
        # Другий Dropout шар для регуляризації
        tf.keras.layers.Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)),
        # Вихідний шар
        tf.keras.layers.Dense(1, activation="linear")
    ])

    # Компіляція моделі з функцією втрат 'mse' та оптимізатором 'adam'
    model.compile(optimizer='adam', loss='mse')
    return model

tuner = BayesianOptimization(
    build_model,
    objective='val_loss',  # мета - мінімізувати валідаційну втрату
    max_trials=20,  # максимальна кількість спроб для пошуку найкращих гіперпараметрів
    directory='tuner_directory',  # директорія для збереження результатів
    project_name='lstm_hyperparameter_tuning'  # назва проекту
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))  # пошук
best_hyperparam = tuner.get_best_hyperparameters(1)[0]  # найкращі гіперпараметри
#print(best_hyperparam)

# Створення фінальної моделі з найкращими гіперпараметрами
# final_model = build_model(best_hp)
# Тренування фінальної моделі
# final_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Trial 20 Complete [00h 01m 29s]
val_loss: 0.7280365824699402

Best val_loss So Far: 0.7280365824699402
Total elapsed time: 00h 15m 04s


In [ ]:
print(best_hyperparam)

In [ ]:
print(best_hyperparam['units'], best_hyperparam['dropout'], best_hyperparam['units2'], best_hyperparam['dropout2'])

256 0.2 256 0.2


## LSTM модель з підібраними гіперпараметрами

Передамо найкращі гіперпараметри у відповідних місцях при побудові моделі:

In [ ]:
model_LSTM = tf.keras.models.Sequential([
    # Перший шар RNN з налаштуванням кількості одиниць як найкращої
    tf.keras.layers.RNN(
        tf.keras.layers.LSTMCell(best_hyperparam['units'], implementation=2),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    # Dropout шар для регуляризації
    tf.keras.layers.Dropout(best_hyperparam['dropout']),
    # Другий шар RNN з налаштуванням кількості одиниць як найкращої
    tf.keras.layers.RNN(
        tf.keras.layers.LSTMCell(best_hyperparam['units2'], implementation=2)),
    # Другий Dropout шар для регуляризації
    tf.keras.layers.Dropout(best_hyperparam['dropout2']),
    # Вихідний шар
    tf.keras.layers.Dense(1, activation="linear")
])

model_LSTM.compile(optimizer='adam', loss='mse')

Тренуємо:

In [ ]:
train_model(model_LSTM, epochs=120)

Epoch 1/120
124/124 [==============================] - 6s 34ms/step - loss: 80.9941 - val_loss: 8.0606
Epoch 2/120
124/124 [==============================] - 6s 48ms/step - loss: 6.1525 - val_loss: 3.3791
Epoch 3/120
124/124 [==============================] - 6s 52ms/step - loss: 3.6932 - val_loss: 4.6454
Epoch 4/120
124/124 [==============================] - 8s 67ms/step - loss: 3.0716 - val_loss: 2.0929
Epoch 5/120
124/124 [==============================] - 7s 59ms/step - loss: 2.8043 - val_loss: 2.1114
Epoch 6/120
124/124 [==============================] - 10s 77ms/step - loss: 2.4090 - val_loss: 0.9869
Epoch 7/120
124/124 [==============================] - 7s 57ms/step - loss: 2.2061 - val_loss: 0.8455
Epoch 8/120
124/124 [==============================] - 6s 52ms/step - loss: 2.2187 - val_loss: 1.1729
Epoch 9/120
124/124 [==============================] - 8s 68ms/step - loss: 2.0523 - val_loss: 0.8980
Epoch 10/120
124/124 [==============================] - 6s 51ms/step - loss: 1.8

Валідація:

In [ ]:
validate_model(model_LSTM)

31/31 [==============================] - 0s 8ms/step - loss: 0.5511
Test loss: 0.5511458516120911


Графік:

In [ ]:
plot_predictions(model_LSTM)

31/31 [==============================] - 1s 9ms/step
        Actual  Predicted        Date
972   3.859375   3.726032  2000-08-14
224   3.945312   3.944346  2000-08-22
722   4.078125   4.049009  2000-09-05
260   4.187500   4.045255  2000-09-06
877   4.218750   4.113012  2000-09-11
..         ...        ...         ...
748  24.250000  24.840910  2020-03-04
298  15.730000  22.824217  2020-03-06
631  15.080000  16.846230  2020-03-10
849  14.380000  13.395309  2020-03-12
46   10.410000  10.208023  2020-03-25

[989 rows x 3 columns]


# 1D-CNN-LSTM-DNN Модель

## Підбір найкращих гіперпараметрів

In [ ]:
def build_model(hp):
    model_1d_cnn_lstm = tf.keras.models.Sequential([
        # згортковий 1D шар
        tf.keras.layers.Conv1D(filters=hp.Int('filters', min_value=16, max_value=256, step=16), kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        tf.keras.layers.MaxPooling1D(2),
        # шар RNN
        tf.keras.layers.RNN(tf.keras.layers.LSTMCell(hp.Int('units', min_value=16, max_value=1024, step=64), implementation=2), return_sequences=True),
        # Dropout для регулятизації
        tf.keras.layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)),
        tf.keras.layers.Flatten(),
        # повнозв'язний шар
        tf.keras.layers.Dense(units=hp.Int('dense_units', min_value=32, max_value=256, step=16), activation='relu'),
        # другий Dropout
        tf.keras.layers.Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)),
        # вихідний шар
        tf.keras.layers.Dense(units=1)
    ])

    model_1d_cnn_lstm.compile(optimizer='adam', loss='mse')
    return model_1d_cnn_lstm


tuner = BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=30,
    directory='tuner_directory',
    project_name='1d_cnn_lstm_hyperparameter_tuning'
)


tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

best_hyperparam = tuner.get_best_hyperparameters(1)[0]

Trial 30 Complete [00h 01m 25s]
val_loss: 0.765660285949707

Best val_loss So Far: 0.6755934953689575
Total elapsed time: 00h 39m 45s


In [ ]:
print(best_hyperparam['filters'], best_hyperparam['units'], best_hyperparam['dropout'], best_hyperparam['dense_units'],best_hyperparam['dropout2'])

32 592 0.30000000000000004 112 0.2


In [ ]:
best_hyperparam

## 1D-CNN-LSTM-DNN модель з підібраними гіперпараметрами

In [ ]:
model_1d_cnn_lstm_dnn = tf.keras.models.Sequential([
        # згортковий 1D шар
        tf.keras.layers.Conv1D(filters=best_hyperparam['filters'], kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        tf.keras.layers.MaxPooling1D(2),
        # шар RNN
        tf.keras.layers.RNN(tf.keras.layers.LSTMCell(best_hyperparam['units'], implementation=2), return_sequences=True),
        # Dropout для регулятизації
        tf.keras.layers.Dropout(best_hyperparam['dropout']),
        tf.keras.layers.Flatten(),
        # повнозв'язний шар
        tf.keras.layers.Dense(units=best_hyperparam['dense_units'], activation='relu'),
        # другий Dropout
        tf.keras.layers.Dropout(best_hyperparam['dropout2']),
        # вихідний шар
        tf.keras.layers.Dense(units=1)
    ])

In [ ]:
model_1d_cnn_lstm_dnn.compile(optimizer='adam', loss='mse')

Тренуємо:

In [ ]:
train_model(model_1d_cnn_lstm_dnn, epochs=100)

Epoch 1/100
124/124 [==============================] - 8s 39ms/step - loss: 66.2026 - val_loss: 3.0207
Epoch 2/100
124/124 [==============================] - 4s 32ms/step - loss: 6.8671 - val_loss: 1.4134
Epoch 3/100
124/124 [==============================] - 4s 31ms/step - loss: 6.4391 - val_loss: 1.6617
Epoch 4/100
124/124 [==============================] - 4s 32ms/step - loss: 5.6577 - val_loss: 1.2660
Epoch 5/100
124/124 [==============================] - 4s 35ms/step - loss: 5.4208 - val_loss: 0.8135
Epoch 6/100
124/124 [==============================] - 4s 32ms/step - loss: 5.6800 - val_loss: 0.8367
Epoch 7/100
124/124 [==============================] - 4s 31ms/step - loss: 5.0333 - val_loss: 0.8664
Epoch 8/100
124/124 [==============================] - 4s 31ms/step - loss: 5.1041 - val_loss: 0.7718
Epoch 9/100
124/124 [==============================] - 4s 35ms/step - loss: 5.2336 - val_loss: 1.0398
Epoch 10/100
124/124 [==============================] - 4s 33ms/step - loss: 5.57

Валідація:

In [ ]:
validate_model(model_1d_cnn_lstm_dnn)

31/31 [==============================] - 0s 7ms/step - loss: 0.6768
Test loss: 0.6768060922622681


Графік:

In [ ]:
plot_predictions(model_1d_cnn_lstm_dnn)

31/31 [==============================] - 0s 7ms/step
        Actual  Predicted        Date
972   3.859375   3.815485  2000-08-14
224   3.945312   3.991898  2000-08-22
722   4.078125   4.066468  2000-09-05
260   4.187500   4.096891  2000-09-06
877   4.218750   4.215067  2000-09-11
..         ...        ...         ...
748  24.250000  24.188604  2020-03-04
298  15.730000  23.017973  2020-03-06
631  15.080000  13.365528  2020-03-10
849  14.380000  13.393053  2020-03-12
46   10.410000   9.718760  2020-03-25

[989 rows x 3 columns]
